<a href="https://colab.research.google.com/github/Ul-tima/ChatGPTCoverLetter/blob/main/ChatGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install  --quiet openai python-dotenv gradio

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

%cd /content/gdrive/MyDrive/ChatGPT/

Mounted at /content/gdrive
/content/gdrive/MyDrive/ChatGPT


In [ ]:
import openai
import os
import re
import urllib.request
from urllib.parse import urlparse
from bs4 import BeautifulSoup


from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo", temperature=0.7):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    return response.choices[0].message["content"]

In [ ]:
class Job:
  def __init__(self, url):
    response = urllib.request.urlopen(url)
    self.soup = BeautifulSoup(response,'html.parser',
                         from_encoding=response.info().get_param('charset'))
    self.title = self.soup.find('div', class_='detail--title-wrapper').get_text().strip()
    self.company = self.soup.find('a', class_='job-details--title').get_text().strip()
    self.recruiter = self.soup.find('a', class_='job-details--recruiter-name').get_text().strip()
    self.description = self.soup.find('div', class_='profile-page-section').get_text().strip()


In [ ]:
class CoverLetter:
    def __init__(self, cv, description, recruiter='', title='', company=''):
        self.response=None
        self.creativity=0.7
        self.cv = cv
        self.description = description
        self.recruiter = recruiter
        self.title = title
        self.company = company

    def set_creativity(self, value):
        if value <=1 and value >0:
          self.creativity=value
        else:
          print("Creativity indicator should be between 0 and 1. Recommended values are in the range of 0.5-0.7")

    def generete(self):
        prompt = f"""
        Write a cover letter with only 200 words in English to recruter {self.recruiter} using the next instructions:
        1. Read my CV and job description which are delimited with triple backticks.
        2. Define company name, project description, responsibilities and requirements for {self.title} position.
        3. Determine what skills and expertise are mentioned in the CV and required in the job description.
        4. Choose the most relevant duties and experience at work and during other projects based on a comparison of those indicated in the vacancy and in the CV.
        5. Based on previous steps write a cover letter \
        that highlights why the candidate is suitable for the role, \
        focusing on skills and enthusiasm for the industry. \
        First paragraph should sign off with my excitement for the role and give specific reasons why I'm interested in joining company. \
        Avoid duplicating information from the CV.\
        Select 3 most relevant examples from most recent experiences and briefly illustrate how those skills were applied. \
        Include a mix of technical and soft skills.\
        Incorporate only the technologies and frameworks mentioned in the CV and required in the job description from step 3.\
        Write in a professional, friendly and upbeat tone. \
        Make it short.
        6. Show only cover letter.
        CV: '''{self.cv}'''
        Job description: '''{self.description}'''
        """
        self.response = get_completion(prompt, temperature = self.creativity)
        return self.response
    def save_to_file(self, filename=None):
        if filename is None:
          filename =" ".join(["letter", self.title, self.company,".txt"] )
        filename = re.sub(r'[\\/*?:"<> |]',"_",filename)
        with open(filename, 'a') as f:
            f.write('\n\n')
            f.write(self.response)

In [ ]:
cv = open("Julia_Pikovets_DS.txt", "r").read()

url = 'https://djinni.co/jobs/572624-chatgpt-openai-rd-engineer-data-science-/'
job = Job(url)

cover_letter = CoverLetter(cv, job.description, job.recruiter, job.title, job.company)
cover_letter.generete()
cover_letter.save_to_file()

In [ ]:
import gradio as gr


def generate(CV, url, creativity):
    job = Job(url)
    cover_letter = CoverLetter(CV, job.description, job.recruiter, job.title, job.company)
    cover_letter.set_creativity(creativity)
    letter = cover_letter.generete()
    cover_letter.save_to_file()
    return letter

demo = gr.Interface(
    fn=generate,
    inputs=[gr.Textbox(lines=10, placeholder="CV here..."), gr.Textbox(lines=1, placeholder="Link to the job on Djinni"), gr.Slider(0.1, 1, 0.7)],
    outputs=gr.outputs.Textbox(label="Cover Letter"),
    allow_flagging="never",
    title = 'Cover Letter Generator',
    description='The demo utilizes ChatGPT to generate cover letter on a CV and a URL for the job description on the Djinni website'

)

demo.launch(share=True)

/usr/local/lib/python3.10/dist-packages/gradio/outputs.py:22: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://a0fbd21b56281f5799.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
